large scale data processing 
use Python, Scala, or Java to process data

scalable, fast, directed acyclic graph to optimize workflows

MLLib - machine learning
Spark SQL
GraphX
Spark Streaming 

# Resilent Distributed Dataset
handle failure

store valued inforamtion 

## Spark code in Python:
find the worst rated movie

In [ ]:
#spark-submit ...py (in terminal)

In [ ]:
from pyspark import SparkConf, SparkContext

def loadMovieNames():
    '''
    open the file and convert each line into a dictionary object with {movie id: moviename}
    '''
    movieNames = {}
    with open("ml-100k/u.iten") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(field[0])] = fields[1]
    return movieNames

def parseInput(line):
    '''
    convert each line into a  object with (movie id,(rating, 1.0))
    '''
    fields=line.split()
    return (int(fields[1]),(float(fields[2]),1.0))

if __name__ == "__main__":
    #create spark context
    conf = SparkConf().setAppName("worstmovies")
    sc = SparkContext(conf = conf)
    
    movieNames = loadMovieNames()
    
    #load up the raw u.data file
    lines = sc.textfile("...")
    
    movieRatings = lines.map(parseInput)
    
    #sum of ratings, totoal ratings
    ratingTotalsAndCount = movieRatings.reduceByKey(lambda movie1, movie2: (movie1[0] + movie2[0], movie1[1] + movie2[1] ) )
    
    averageRatings = ratingTotalsAndCount.mapValues(lambda totalAndCount:totalAndCount[0] / totalAndCount[1])
    
    sortedMovies = averageRatings.sortBy(lambda x:x[1])
    
    results = sortedMovies.take(10)



### find the worst rated movie that is rated by at least 10 people

In [ ]:
from pyspark import SparkConf, SparkContext

def loadMovieNames():
    '''
    open the file and convert each line into a dictionary object with {movie id: moviename}
    '''
    movieNames = {}
    with open("ml-100k/u.iten") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(field[0])] = fields[1]
    return movieNames

def parseInput(line):
    '''
    convert each line into a  object with (movie id,(rating, 1.0))
    '''
    fields=line.split()
    return (int(fields[1]),(float(fields[2]),1.0))

if __name__ == "__main__":
    #create spark context
    conf = SparkConf().setAppName("worstmovies")
    sc = SparkContext(conf = conf)
    
    movieNames = loadMovieNames()
    
    #load up the raw u.data file
    lines = sc.textfile("...")
    
    movieRatings = lines.map(parseInput)
    
    ratingTotalsAndCount = movieRatings.reduceByKrk(lambda movie1, movie2: (movie1[0] + movie2[0], movie1[1] + movie2[1] ) )
    
    #(movie id,(rating, 1.0)) x[1]=(sum of rating, count)
    populartotalsandcount = ratingTotalsAndCount.filter(lambda x:[1][1] > 10)
    
    averageRatings = populartotalsandcount.mapValues(lambda totalAndCcount:totalAndCount[0] / totalAndCount[1])
    
    sortedMovies = averageRatings.sortBy(lambda x:x[1])
    
    results = sortedMovies.take(10)




## SparkSQL in Python

In [ ]:
from pyspark.sql import SQLContext, Row

hiveContext=HiveContext(sc)
inputData=spark.read.json(dataFile)
inputData.createOrReplaceTempView("myStructuredStuffed")
#create a database table 
myResultDataFrame=hiveContextsql('''
                                SELECT foo FROM bar ORDER BY foobar
                                ''')

In [ ]:
myResultDataFrame.show()
select filter groupby rdd()

## Spark2

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

def loadMovieNames():
	movieNames = {}
	with open('ml-100k/u.item') as f:
		for line in f:
			fields = line.split('|')
			movieNames[int(fields[0])] = fields[1]
	return movieNames

def parseInput(line):
	fields = line.split()
	return Row(movieID = int(fields[1]), rating = float(fields[2]))

if __name__ == "__main__":
	# Create a SparkSession
	spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

	# load up our movie ID -> name dictionary
	movieNames = loadMovieNames()

	# get the raw data
	lines = spark.sparkContext.textFile("hdfs:///user/maria_dev/ml-100k/u.data")

	# Convert it to a RDD of Row objects with (movieID, rating)
	movies = lines.map(parseInput)

	# Convert that to a DataFrame
	movieDataset = spark.createDataFrame(movies)

	# Compute counts of ratings for each movieID
	counts = movieDataset.groupBy("movieID").count()

	# Compute average rating for each movieID
	averageRatings = movieDataset.groupBy("movieID").avg("rating")

	# Join the two together (We now have movieID, avg(rating), and count columns)
	averagesAndCounts = counts.join(averageRatings, "movieID")

	# Pull the top 10 results
	topTen = averagesAndCounts.orderBy("avg(rating)").take(10)

    # Print them out, converting movie ID's to names as we go.
    for movie in topTen:
   		print (movieNames[movie[0]], movie[1], movie[2])

    # Stop the session


### find the worst rated movie that is rated by at least 10 people

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions

def loadMovieNames():
	movieNames = {}
	with open('ml-100k/u.item') as f:
		for line in f:
			fields = line.split('|')
			movieNames[int(fields[0])] = fields[1]
	return movieNames

def parseInput(line):
	fields = line.split()
	return Row(movieID = int(fields[1]), rating = float(fields[2]))

if __name__ == "__main__":
	# Create a SparkSession
	spark = SparkSession.builder.appName("PopularMovies").getOrCreate()

	# load up our movie ID -> name dictionary
	movieNames = loadMovieNames()

	# get the raw data
	lines = spark.sparkContext.textFile("hdfs:///user/maria_dev/ml-100k/u.data")

	# Convert it to a RDD of Row objects with (movieID, rating)
	movies = lines.map(parseInput)

	# Convert that to a DataFrame
	movieDataset = spark.createDataFrame(movies)

	# Compute counts of ratings for each movieID
	counts = movieDataset.groupBy("movieID").count()

	# Filter the count of ratings for movies < 10 or fewer times
	filteredCounts = counts.filter("count > 10")

	# Compute average rating for each movieID
	averageRatings = movieDataset.groupBy("movieID").avg("rating")

	# Join the two together (We now have movieID, avg(rating), and count columns)
	averagesAndCounts = filteredCounts.join(averageRatings, "movieID")

	# Pull the top 10 results
	topTen = averagesAndCounts.orderBy("avg(rating)").take(10)

    # Print them out, converting movie ID's to names as we go.
    for movie in topTen:
   		print (movieNames[movie[0]], movie[1], movie[2])

    # Stop the session


# Use ML-ALS to recommend movies

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import lit

# Load up movie ID -> movie name dictionary
def loadMovieNames():
    movieNames = {}
    with open("ml-100k/u.item") as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1].decode('ascii', 'ignore')
    return movieNames

# Convert u.data lines into (userID, movieID, rating) rows
def parseInput(line):
    fields = line.value.split()
    return Row(userID = int(fields[0]), movieID = int(fields[1]), rating = float(fields[2]))


if __name__ == "__main__":
    # Create a SparkSession (the config bit is only for Windows!)
    spark = SparkSession.builder.appName("MovieRecs").getOrCreate()

    # Load up our movie ID -> name dictionary
    movieNames = loadMovieNames()

    # Get the raw data 
    lines = spark.read.text("hdfs:///user/maria_dev/ml-100k/u.data").rdd

    # Convert it to a RDD of Row objects with (userID, movieID, rating)
    ratingsRDD = lines.map(parseInput)

    # Convert to a DataFrame and cache it
    ratings = spark.createDataFrame(ratingsRDD).cache()

    # Create an ALS collaborative filtering model from the complete data set
    als = ALS(maxIter=5, regParam=0.01, userCol="userID", itemCol="movieID", ratingCol="rating")
    model = als.fit(ratings)

    # Print out ratings from user 0:
    print("\nRatings for user ID 0:")
    userRatings = ratings.filter("userID = 0")
    for rating in userRatings.collect():
        print movieNames[rating['movieID']], rating['rating']

    print("\nTop 20 recommendations:")
    # Find movies rated more than 100 times
    ratingCounts = ratings.groupBy("movieID").count().filter("count > 100")
    # Construct a "test" dataframe for user 0 with every movie rated more than 100 times
    popularMovies = ratingCounts.select("movieID").withColumn('userID', lit(0))

    # Run our model on that list of popular movies for user ID 0
    recommendations = model.transform(popularMovies)

    # Get the top 20 movies with the highest predicted rating for this user
    topRecommendations = recommendations.sort(recommendations.prediction.desc()).take(20)

    for recommendation in topRecommendations:
        print (movieNames[recommendation['movieID']], recommendation['prediction'])

    spark.stop()